Imports

In [1]:
from __future__ import print_function
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.autograd import Variable
from typing import Tuple
from torch.utils.data import Dataset,DataLoader
from sklearn.metrics import confusion_matrix, top_k_accuracy_score
import torchvision                                                       
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torchvision.transforms import ToTensor
from numpy import random as rd
import torch
import torch.nn as nn
import torch.optim as optim

Residual Block

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride = stride, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride = 1, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels*2, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(out_channels*2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride = stride, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride = 1, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride = 1, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels*2, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(out_channels*2)
        )
        
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        # out = self.conv1(x)
        # out = self.conv2(out)
        out = torch.cat([self.conv1(x), self.conv2(x)], 1)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

Renset Block

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1))

In [ ]:
class Resnet(nn.Module):
    def __init__(self, block, blockList) -> None:
        super(Resnet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(inChannels=1, outChannels=64, kernelSize=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(3, 2, 1)
        )
        
        # 3, 3, 3, 3 blocks
        self.block0 = self._make_layer(block,   inChannels=64,  inChannels=64, blocksNum=blockList[0], stride=1)
        self.block1 = self._make_layer(block,  inChannels=256, inChannels=128, blocksNum=blockList[1], stride=1)
        self.block2 = self._make_layer(block,  inChannels=512, inChannels=256, blocksNum=blockList[2], stride=1)
        self.block3 = self._make_layer(block, inChannels=1024, inChannels=512, blocksNum=blockList[3], stride=2)

        # apply 2D adpative average pooling from 1 input to 1 plane
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        # flatten the data into 1 dimension
        self.flatten = nn.Flatten()
        # apply dropout to output with 60% percent chance
        self.drop = nn.Dropout(0.6)
        # connect 2048 input nodes into 10 output nodes
        self.fc = nn.Linear(2048, 10)

    # helper function that adds layer by layer along with the res block
    def _make_layer(self, block, inChannels, outChannels, blocksNum, stride):
        layers = []
        layers.append(block(inChannels, outChannels, stride))

        inChannels = outChannels * block.expansion
        for _ in range(1, blocksNum):
            layers.append(block(inChannels, outChannels, 1))

        return nn.Sequential(*layers)
    
    # forward function 
    def forward(self, x):
        x = self.conv1(x)
        x = self.block0(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.avgpool(x)
        x = self.drop(x)
        x = self.fc(x)
        return x